In [8]:
import geopandas as gpd
import pandas as pd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon

%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


import networkx as nx

In [19]:
def is_geohash_in_bounding_box(current_geohash, bbox_coordinates):
    """Checks if the box of a geohash is inside the bounding box

    :param current_geohash: a geohash
    :param bbox_coordinates: bounding box coordinates
    :return: true if the the center of the geohash is in the bounding box
    """

    coordinates = geohash.decode(current_geohash)
    geohash_in_bounding_box = (bbox_coordinates[0] < coordinates[0] < bbox_coordinates[2]) and (
            bbox_coordinates[1] < coordinates[1] < bbox_coordinates[3])
    return geohash_in_bounding_box
def build_geohash_box(current_geohash):
    """Returns a GeoJSON Polygon for a given geohash

    :param current_geohash: a geohash
    :return: a list representation of th polygon
    """

    b = geohash.bbox(current_geohash)
    polygon = [(b['w'], b['s']), (b['w'], b['n']), (b['e'], b['n']), (b['e'], b['s'],), (b['w'], b['s'])]
    return polygon
def compute_geohash_tiles(bbox_coordinates,GEOHASH_PRECISION):
    """Computes all geohash tile in the given bounding box

    :param bbox_coordinates: the bounding box coordinates of the geohashes
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding box, assuming the earth is flat ;)
    center_latitude = (bbox_coordinates[0] + bbox_coordinates[2]) / 2
    center_longitude = (bbox_coordinates[1] + bbox_coordinates[3]) / 2

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            if neighbor not in checked_geohashes and is_geohash_in_bounding_box(neighbor, bbox_coordinates):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes
def create_polygon(coordinates):
    polygon_geom = Polygon(coordinates)
    crs = {'init': 'epsg:4326'}
    polygon = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])     
    return polygon.geometry # return polygon['geometry']
# this function is for save the geohashes into a json file 
def write_geohash_layer(geohashes,filename):
    """Writes a grid layer based on the geohashes

    :param geohashes: a list of geohashes
    """

    layer = MultiLineString([build_geohash_box(gh) for gh in geohashes])
    with open('result/' + filename + '.json', 'wb') as f:
        f.write(geojson.dumps(layer,sort_keys=True).encode('utf-8'))

def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    
    #center_latitude = polygon.centroid.coords[0][0]
    #center_longitude = polygon.centroid.coords[0][1]
    
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes

def getNewDistance(lat1,lon1,degree,radius):
    R = 6378.1 #Radius of the Earth
    brng = math.radians(degree) #Bearing is 90 degrees converted to radians.
    d = radius #Distance in km

    #lat2  52.20444 - the lat result I'm hoping for
    #lon2  0.36056 - the long result I'm hoping for.

    lat1 = math.radians(lat1) #Current lat point converted to radians
    lon1 = math.radians(lon1) #Current long point converted to radians

    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1), math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    print(lat2)
    print(lon2)
    return lat2,lon2


def getDistance(lat1, long1, distance, bearing):
    
    try:
        # Define starting point.
        start = geopy.Point(lat1, long1)

        # Define a general distance object, initialized with a distance of 1 km.
        d = geopy.distance.VincentyDistance(kilometers = distance)

        # Use the `destination` method with a bearing of 0 degrees (which is north)
        # in order to go from point `start` 1 km to north.
        #print d.destination(point=start, bearing=0)
        return d.destination(point=start,bearing=bearing)
    except:
        pass

def quadraticpolygon(c,a,b):
    try:
        a = list(a)
        b = list(b)
        c = list(c)
        coords = np.array([a,b,c])
        f = scipy.interpolate.interp1d(coords[:, 0], coords[:, 1],kind='quadratic') #, kind='quadratic'
        fine_x = np.linspace(np.min(coords[:, 0]), np.max(coords[:, 0]),20)
        fine_y = f(fine_x)
        fine_coords = zip(fine_x, fine_y)
        #fine_line = LineString(fine_coords)
        myshape = geometry.Polygon(fine_coords)
        return myshape
    except:
        pass

    
def generatePolygon(lat1, long1, radius, angle1, angle2):
    import geopandas as gpd
    from shapely.geometry import Polygon
    lat_point_list = []
    lon_point_list = []
    point1 = getDistance(lat1, long1, radius, angle1)
    point2 = getDistance(lat1, long1, radius, angle2)
    
    point1_lat = point1[0]
    point1_lon = point1[1]
    
    point2_lat = point2[0]
    point2_lon = point2[1]
    
    lat_point_list.append(lat1)
    lat_point_list.append(point1_lat)
    lat_point_list.append(point2_lat)
    
    lon_point_list.append(long1)
    lon_point_list.append(point1_lon)
    lon_point_list.append(point2_lon)
    
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    #print(polygon.geometry)
    
    return polygon.geometry



def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes


def splitDataFrameList(df,target_column,separator):
    import pandas
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pandas.DataFrame(new_rows)
    return new_df

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [88]:
selangor_road = gp.GeoDataFrame.from_file('data/selangor_data/selangor_routes.shp')
selangor_road.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,4931450,5122,residential,Jalan Pinggiran USJ 1/17,None,B,0,0,F,F,...,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55520 3.05020, 101.55625 3.050..."
1,4931468,5115,tertiary,Jalan Pinggiran USJ 1,None,B,0,0,F,F,...,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55488 3.05006, 101.55509 3.050..."
2,4931834,5122,residential,Persiaran Perusahaan U/1,None,B,0,0,F,F,...,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55467 3.09144, 101.55465 3.090..."
3,4931877,5113,primary,Jalan Montford,B9,F,0,0,F,F,...,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55272 3.08739, 101.55219 3.08701)"
4,4934921,5113,primary,Bulatan Megawati,None,F,0,0,F,F,...,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.54352 3.05211, 101.54337 3.052..."


In [12]:
selangor_poi = gp.GeoDataFrame.from_file('data/selangor_data/selangor_poi.shp')
selangor_poi.head()

,osm_id,code,fclass,name,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,19206023,2501,supermarket,Giant,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.51661 3.04433, 101.51697 3.04445..."
1,19206050,2501,supermarket,Mydin,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.51561 3.04517, 101.51576 3.04548..."
2,19207507,2501,supermarket,Tesco Extra,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.54991 3.07746, 101.55124 3.07781..."
3,19207509,2256,stadium,Stadium Malawati,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.54664 3.07990, 101.54664 3.08028..."
4,19207529,2256,stadium,Stadium Shah Alam,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.54516 3.08373, 101.54523 3.08379..."


In [89]:
## Getting the centroid coordinates for lat long
selangor_road['lat'] = selangor_road['geometry'].centroid.y
selangor_road['lon'] = selangor_road['geometry'].centroid.x

selangor_poi['lat'] = selangor_poi['geometry'].centroid.y
selangor_poi['lon'] = selangor_poi['geometry'].centroid.x

In [90]:
selangor_road['geohash'] = selangor_road['geometry'].apply(lambda x:compute_geohash_tiles_from_polygon(x,7))
selangor_poi['geohash'] = selangor_poi['geometry'].apply(lambda x:compute_geohash_tiles_from_polygon(x,7))

In [91]:
selangor_road.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,lat,lon,geohash
0,4931450,5122,residential,Jalan Pinggiran USJ 1/17,None,B,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55520 3.05020, 101.55625 3.050...",3.050336,101.558314,[w281r52]
1,4931468,5115,tertiary,Jalan Pinggiran USJ 1,None,B,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55488 3.05006, 101.55509 3.050...",3.049863,101.557853,[w281r50]
2,4931834,5122,residential,Persiaran Perusahaan U/1,None,B,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55467 3.09144, 101.55465 3.090...",3.088406,101.554642,[w281wfj]
3,4931877,5113,primary,Jalan Montford,B9,F,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55272 3.08739, 101.55219 3.08701)",3.087199,101.552459,[w281wfh]
4,4934921,5113,primary,Bulatan Megawati,None,F,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.54352 3.05211, 101.54337 3.052...",3.052685,101.543664,[w281qet]


In [17]:
selangor_poi.head()

,osm_id,code,fclass,name,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,lat,lon,geohash
0,19206023,2501,supermarket,Giant,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.51661 3.04433, 101.51697 3.04445...",3.044194,101.516851,[w281q44]
1,19206050,2501,supermarket,Mydin,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.51561 3.04517, 101.51576 3.04548...",3.045091,101.515754,[w281q43]
2,19207507,2501,supermarket,Tesco Extra,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.54991 3.07746, 101.55124 3.07781...",3.077160,101.550705,"[w281wb4, w281wb5]"
3,19207509,2256,stadium,Stadium Malawati,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.54664 3.07990, 101.54664 3.08028...",3.080081,101.547192,[w281wb8]
4,19207529,2256,stadium,Stadium Shah Alam,MYS,Malaysia,MYS.15_1,Selangor,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"POLYGON ((101.54516 3.08373, 101.54523 3.08379...",3.082196,101.545894,[w281w9p]


In [22]:
### For poi that has multiple geohash, each geohash would contain the same poi

poi_explode = explode(selangor_poi, ['geohash'])

In [24]:
poi_explode.head()

,CC_1,ENGTYPE_1,GID_0,GID_1,HASC_1,NAME_0,NAME_1,NL_NAME_1,TYPE_1,VARNAME_1,code,fclass,geometry,lat,lon,name,osm_id,geohash
0,None,State,MYS,MYS.15_1,MY.SL,Malaysia,Selangor,None,Negeri,Selangor Darul Ehsan,2501,supermarket,"POLYGON ((101.51661 3.04433, 101.51697 3.04445...",3.044194,101.516851,Giant,19206023,w281q44
1,None,State,MYS,MYS.15_1,MY.SL,Malaysia,Selangor,None,Negeri,Selangor Darul Ehsan,2501,supermarket,"POLYGON ((101.51561 3.04517, 101.51576 3.04548...",3.045091,101.515754,Mydin,19206050,w281q43
2,None,State,MYS,MYS.15_1,MY.SL,Malaysia,Selangor,None,Negeri,Selangor Darul Ehsan,2501,supermarket,"POLYGON ((101.54991 3.07746, 101.55124 3.07781...",3.077160,101.550705,Tesco Extra,19207507,w281wb4
3,None,State,MYS,MYS.15_1,MY.SL,Malaysia,Selangor,None,Negeri,Selangor Darul Ehsan,2501,supermarket,"POLYGON ((101.54991 3.07746, 101.55124 3.07781...",3.077160,101.550705,Tesco Extra,19207507,w281wb5
4,None,State,MYS,MYS.15_1,MY.SL,Malaysia,Selangor,None,Negeri,Selangor Darul Ehsan,2256,stadium,"POLYGON ((101.54664 3.07990, 101.54664 3.08028...",3.080081,101.547192,Stadium Malawati,19207509,w281wb8


In [38]:
poi_geohash = poi_explode.groupby('geohash', as_index=False).count()
poi_geohash.sort_values(by='osm_id',ascending=False).head(15)

,geohash,CC_1,ENGTYPE_1,GID_0,GID_1,HASC_1,NAME_0,NAME_1,NL_NAME_1,TYPE_1,VARNAME_1,code,fclass,geometry,lat,lon,name,osm_id
2797,w281wr1,0,12,12,12,12,12,12,0,12,12,12,12,12,12,12,12,12
1641,w281qp4,0,11,11,11,11,11,11,0,11,11,11,11,11,11,11,11,11
7856,w286dbm,0,10,10,10,10,10,10,0,10,10,10,10,10,10,10,7,10
1284,w281mqz,0,10,10,10,10,10,10,0,10,10,10,10,10,10,10,1,10
4015,w282fw4,0,9,9,9,9,9,9,0,9,9,9,9,9,9,9,4,9
3078,w281xqu,0,9,9,9,9,9,9,0,9,9,9,9,9,9,9,1,9
622,w280h6w,0,8,8,8,8,8,8,0,8,8,8,8,8,8,8,6,8
5443,w2834jz,0,8,8,8,8,8,8,0,8,8,8,8,8,8,8,8,8
2530,w281w0c,0,7,7,7,7,7,7,0,7,7,7,7,7,7,7,7,7
1965,w281tb5,0,7,7,7,7,7,7,0,7,7,7,7,7,7,7,1,7


In [55]:
poi_geohash['geohash'].head()

0    w0xcpvs
1    w0xfztp
2    w0xfztq
3    w0xfztr
4    w0xfztt
Name: geohash, dtype: object

In [96]:
selangor_road['geohash'].head()

0    w281r52
1    w281r50
2    w281wfj
3    w281wfh
4    w281qet
Name: geohash, dtype: object

In [95]:
selangor_road['geohash'] = selangor_road['geohash'].astype(str)
selangor_road['geohash'] = selangor_road['geohash'].str[1:]
selangor_road['geohash'] = selangor_road['geohash'].str[:-1]
selangor_road['geohash'] = selangor_road['geohash'].str[1:]
selangor_road['geohash'] = selangor_road['geohash'].str[:-1]
selangor_road.head()
#print(len(selangor_road))
#print(len(explode(selangor_road, ['geohash'])))

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,lat,lon,geohash
0,4931450,5122,residential,Jalan Pinggiran USJ 1/17,None,B,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55520 3.05020, 101.55625 3.050...",3.050336,101.558314,w281r52
1,4931468,5115,tertiary,Jalan Pinggiran USJ 1,None,B,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55488 3.05006, 101.55509 3.050...",3.049863,101.557853,w281r50
2,4931834,5122,residential,Persiaran Perusahaan U/1,None,B,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55467 3.09144, 101.55465 3.090...",3.088406,101.554642,w281wfj
3,4931877,5113,primary,Jalan Montford,B9,F,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.55272 3.08739, 101.55219 3.08701)",3.087199,101.552459,w281wfh
4,4934921,5113,primary,Bulatan Megawati,None,F,0,0,F,F,...,Selangor Darul Ehsan,None,Negeri,State,None,MY.SL,"LINESTRING (101.54352 3.05211, 101.54337 3.052...",3.052685,101.543664,w281qet


In [97]:
poi_geohash['geohash'] = poi_geohash['geohash'].astype(str)

In [98]:
print(poi_geohash.dtypes)
print(selangor_road.dtypes)

geohash      object
CC_1          int64
ENGTYPE_1     int64
GID_0         int64
GID_1         int64
HASC_1        int64
NAME_0        int64
NAME_1        int64
NL_NAME_1     int64
TYPE_1        int64
VARNAME_1     int64
code          int64
fclass        int64
geometry      int64
lat           int64
lon           int64
name          int64
osm_id        int64
dtype: object
osm_id         object
code            int64
fclass         object
name           object
ref            object
oneway         object
maxspeed        int64
layer           int64
bridge         object
tunnel         object
GID_0          object
NAME_0         object
GID_1          object
NAME_1         object
VARNAME_1      object
NL_NAME_1      object
TYPE_1         object
ENGTYPE_1      object
CC_1           object
HASC_1         object
geometry     geometry
lat           float64
lon           float64
geohash        object
dtype: object


In [99]:
cx_poi_geohash = poi_geohash[['geohash', 'osm_id']]

In [100]:
cx_poi_geohash.sort_values(by='osm_id',ascending=False).head(15)

,geohash,osm_id
2797,w281wr1,12
1641,w281qp4,11
7856,w286dbm,10
1284,w281mqz,10
4015,w282fw4,9
3078,w281xqu,9
622,w280h6w,8
5443,w2834jz,8
2530,w281w0c,7
1965,w281tb5,7


In [101]:
test_final = pd.merge(left=selangor_road, right=cx_poi_geohash, suffixes=('_road','_poi'), on='geohash', how='left')

In [104]:
test_final['osm_id_poi'].fillna(0, inplace=True)

In [105]:
test_final.sort_values(by='osm_id_poi',ascending=False).head(15)


,osm_id_road,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,lat,lon,geohash,osm_id_poi
322,14367451,5122,residential,Jalan Pelapik U8/A,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52616 3.11551, 101.52631 3.115...",3.115138,101.526599,w281wr1,12.0
68649,446763522,5122,residential,Jalan Pelapik U8/B,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52745 3.11528, 101.52709 3.114...",3.115025,101.527247,w281wr1,12.0
1347,23488069,5122,residential,Jalan Pelapik U8/46,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52499 3.11469, 101.52505 3.114...",3.115411,101.526044,w281wr1,12.0
82547,554686184,5141,service,None,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52729 3.11580, 101.52724 3.115...",3.115783,101.527190,w281wr1,12.0
68658,446765541,5155,steps,None,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52709 3.11483, 101.52712 3.114...",3.114796,101.527176,w281wr1,12.0
168483,754819033,5141,service,None,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52692 3.11584, 101.52686 3.115...",3.115905,101.526800,w281wr1,12.0
168484,754819034,5141,service,None,None,B,0,0,F,T,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52709 3.11577, 101.52705 3.115...",3.115800,101.527000,w281wr1,12.0
68656,446765539,5155,steps,None,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52700 3.11472, 101.52699 3.114...",3.114635,101.527040,w281wr1,12.0
68640,446763317,5141,service,None,None,F,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52733 3.11575, 101.52700 3.115...",3.115385,101.527037,w281wr1,12.0
68644,446763436,5122,residential,None,None,F,0,0,F,F,...,None,Negeri,State,None,MY.SL,"LINESTRING (101.52700 3.11534, 101.52644 3.11581)",3.115577,101.526719,w281wr1,12.0


In [106]:
test_final.to_file('results/selangor_roads.shp')